# MNIST Syft Data Owner

In [ ]:
import syft as sy
sy.VERBOSE = False

# PART 1: Launch a Duet Server and Connect

As a data owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server (within this notebook).
We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a data scientist) connect to this server.

You do this by running the code below and following the instructions it gives!

In [ ]:
duet = sy.launch_duet(loopback=True)

If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# PART 2: Launch a Duet Server and Connect

In [ ]:
import torch as th
import torchvision as tv

In [ ]:
duet.store.pandas

In [ ]:
# we need some transforms for our MNIST data set
transform_1 = tv.transforms.ToTensor()  # this converts PIL images to Tensors
transform_2 = tv.transforms.Normalize(0.1307, 0.3081)  # this normalizes the dataset

# compose our transforms
transforms = tv.transforms.Compose([transform_1, transform_2])

In [ ]:
kwargs = {
    "batch_size": 64,
}

use_cuda = th.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
if use_cuda:
    kwargs.update(
            {
                "num_workers": 1,
                "pin_memory": True,
                "shuffle": True
            }
    )
    
private_data = tv.datasets.MNIST('../data', train=True, download=True, transform=transform_1)
private_loader = th.utils.data.DataLoader(private_data,**kwargs)

In [ ]:
# Lets add a tag and description to our dataset so that our Duet partner knows what it is
private_loader = private_loader.tag("#MNIST_private_training_data")
private_loader = private_loader.describe("My super secret MNIST dataset for training the best model!")

In [ ]:
# now lets host it on duet so our partner can access a pointer to the data
# IMPORTANT: The data cannot be viewed, downloaded, or copied without your partner requesting permission

# TODO: This cant be sent currently, how can we share it?
private_loader_ptr = private_loader.send(duet, searchable=True)

In [ ]:
duet.requests.pandas

In [ ]:
duet.requests[0].approve()

In [ ]:
duet.store